# Supplemental Code for "Transductive conformal inference with adaptive scores" by U. Gazin, G. Blanchard, E. Roquain.


## Disclaimer on the packages

You need to download the four following packages before running the code. 
Files jdot.py, classif.py, procedure.py and algo.py are not from this paper. 

jdot.py and classif .py can be found in the github page https://github.com/rflamary/JDOT/tree/master and is the implementation of the methods proposed by N. Courty, R. Flamary, A. Habrard, A. Rakotomamonjy, in "Joint Distribution Optimal Transportation for Domain Adaptation" published in Neural Information Processing Systems (NIPS), 2017.

procedure.py and algo.py can be found in the github page https://github.com/arianemarandon/adadetect#machine-learning-meets-fdr for the implementation and is the implementation of the methods proposed by Ariane Marandon, Lihua Lei, David Mary and Etienne Roquain in the paper "Adaptive novelty detection with false discovery rate guarantee", to appear in Annals of Statistics.

In [ ]:
import random
import numpy as np
import scipy.stats as stat
import statsmodels as sm

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.datasets import fetch_openml 
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.svm import OneClassSVM

In [ ]:
### Procedure of Transfer Learning from:
###          N. Courty, R. Flamary, A. Habrard, A. Rakotomamonjy, 
###         "Joint Distribution Optimal Transportation for Domain Adaptation"

# Go to https://github.com/rflamary/JDOT/tree/master for the implementation

from jdot import jdot_krr
gamma=1

In [ ]:
### AdaDetect Procedure from:
###          Ariane Marandon, Lihua Lei, David Mary and Etienne Roquain,
###          "Machine learning meets false discovery rate"

# Go to https://github.com/arianemarandon/adadetect#machine-learning-meets-fdr for the implementation. 



from procedure import AdaDetectERM ,AdaDetectDE
from algo import BH, EmpBH, adaptiveEmpBH, compute_pvalue

In [ ]:
#Defining OneClass Classifier

class OCC(object):
    
    def __init__(self, scoring_fn):
        self.null = scoring_fn 
    
    def fit(self, x_train, x_null_train):
        self.null.fit(x_null_train)
    
    def score_samples(self, x): 
        return -self.null.score_samples(x) 


In [ ]:
#Setting colors for Graphics

colors_blindness = sns.color_palette("colorblind")

color_train = colors_blindness[8]
color_cal = colors_blindness[4]
color_test = colors_blindness[7] #(0,0,0) #colors_blindness[4]


plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{bm} \usepackage{amsfonts}')

# Simulation of conformal $p$-values: $P_{n,m}$.

In [ ]:
def Distrib_EmpiricalPvalues(n):
    U=stat.dirichlet.rvs(alpha=np.ones(n+1))[0]
    Val=(np.arange(n+1)+1)/(n+1)
    

    DistribCond=stat.rv_discrete(values=(Val,U))
    return DistribCond

def Simu_EmpiricalPvalues(n,m,N=1):
    
    T=np.zeros((N,m))
    for i in range(N):
        DistribCond=Distrib_EmpiricalPvalues(n)
        T[i]=DistribCond.rvs(size=m)

    return T

## Illustration of the Pólya urn model

In [ ]:
def ConditionnalLaw(l,VectCount):
    Values=np.arange(1,l+2)
    ProLaw=(VectCount+1)/(np.sum(VectCount)+(l+1))
    return stat.rv_discrete(name='Conditionnal Law',values=(Values,ProLaw))


def UpdatingPValues(l,VectCount,UpdatePValue=False,P=np.array([])):
    NewPValueLaw=ConditionnalLaw(l,VectCount)
    p=NewPValueLaw.rvs(size=1)
    
    VectCount[p-1]+=1
    
    if UpdatePValue:
        P.append(p)
    
    return(VectCount,P)

In [ ]:
def GraphicalRepresentation(l,VectCount,HightBar=False,m=0,SaveFig=False):
    x=np.arange(1,l+2)
    t=int(np.sum(VectCount))
    tStr=str(t)
    tPlus1Str=str(t+1)
    LabBar=np.empty(l+1,dtype=object)
 
    for i in range(1,l+2):
        s=str(i)+"/"+str(l+1)
        LabBar[i-1]=s
    
    if HightBar:
        plt.ylim(0,m)
    
    ll=np.linspace(1-0.8,l+1+0.8,l+2)
    
    plt.plot(ll,np.ones(l+2),'r')
    
    plt.bar(x,np.ones(l+1)+VectCount,tick_label=LabBar)
    
    plt.title("Unnormalised histogram of $p_{%s}$ conditionnaly on $p_1,\cdots,p_{%s}$" %(tPlus1Str,tStr))
    plt.xlabel("Values of $p_{%s}$"%(tPlus1Str) )
    plt.ylabel("Unnormalised probability of being equal to $x$")
    #plt.legend()
    if SaveFig:
        plt.savefig('Dynamical_'+str(t)+'.pdf',format='pdf')
    
    plt.show()


In [ ]:
def GraphicalEvolution(l,m,ConstantGraph=False,SaveFig=False):
    VectCount=np.zeros(l+1)
    
    if ConstantGraph:
        plt.ylim(0,m+1)
        
    x=np.arange(1,l+2)
    ll=np.linspace(1-0.8,l+1+0.8,l+2)
    
    LabBar=np.empty(l+1,dtype=object)
 
    for i in range(1,l+2):
        s=str(i)+"/"+str(l+1)
        LabBar[i-1]=s
    
    if ConstantGraph:
        plt.ylim(0,m)
    
    plt.bar(x,np.ones(l+1)+VectCount,tick_label=LabBar)
    plt.plot(ll,np.ones(l+2),'r')
    plt.title(r"Unnormalised histogram of $p_1$")
    plt.xlabel(r"Values of $p_1$" )
    plt.ylabel("Unnormalised probability of being equal to $x$")
    #plt.legend(bbox_to_anchor=(1.05, 1.0),loc='upper left')
    
    
    
    if SaveFig:
        plt.savefig('Dynamical_0.pdf',format='pdf')
    
    #plt.legend()
    plt.show()
    for i in range(1,m):
        (VectCount,P)=UpdatingPValues(l,VectCount)
        GraphicalRepresentation(l,VectCount,ConstantGraph,m,SaveFig)
    

In [ ]:
n=5
m=6

In [ ]:
GraphicalEvolution(n,m,ConstantGraph=True)

# Some generalist code for Conformal Prediction

In [ ]:
#The residual Score Function

def my_score(y_pred, y):
    
    """
    Compute the residual in order to compute the score
    
    y_pred: a predicted value of y by any method
    y: the true value of y (for calibration scores) or the possible value y (for the test scores.)
    
    Return: the residual of y and y_pred
    """
    
    return np.abs(y_pred - y)  

In [ ]:
# Create the "conformal" distribution of the calibration set in 

def ConformalDistrib(y_pred_cal,y_cal,n_cal,my_score):
    """
    
    """
    
    residuals_cal = my_score(y_pred_cal,y_cal) 
    residuals_calPlusInfini=np.append(residuals_cal,np.inf)

    Residual_Distrib=stat.rv_discrete(values=np.array([residuals_calPlusInfini,np.ones(n_cal+1)/(n_cal+1)]))
    
    return Residual_Distrib


def ClassicalSCP(y_pred_test,Residual_Distrib,Level):
    quantile_scp = Residual_Distrib.ppf(1-Level)
    ConformalIntervalUp=y_pred_test+quantile_scp
    ConformalIntervalDown=y_pred_test-quantile_scp
    
    return (ConformalIntervalUp,ConformalIntervalDown,quantile_scp)

In [ ]:
def NbCoverage(y_test,ConformalIntervalUp,ConformalIntervalDown):
    return np.sum(((ConformalIntervalDown<=y_test)*(y_test<=ConformalIntervalUp)))

# The new DKW Inequality.

In [ ]:
nIter=8

def IterLambda_Tau(n,m,nIter=8):
    def lambDKW(delta):
        lamb=1
        
        t_nm=n*m/(n+m)
        A1=np.log(1/delta)
        
        #A2=np.sqrt(2*np.pi*(n+m))
        
        A2=np.sqrt(2*np.pi)*2*t_nm/np.sqrt(n+m)
        
        i=-1
        while i<nIter:
            i+=1
            Num=A1+np.log(1+A2*(lamb))
            lamb=np.minimum(np.sqrt(Num/(2*t_nm)),1)
        
        return lamb
    return lambDKW

# Transfer Learning: Code for the comparison of the Methods

## Creating Prediction Interval

In [ ]:
# Defining Regression function

def fNotNoise(x):
    return np.cos(x)

def fAndNoise(x,sigma=0.1):
    n=len(x)
    return fNotNoise(x)+sigma*stat.norm.rvs(size=n)

# Definig the transfer function for calibration and test samples

def Psi(x):
    return 0.6*x+x**(2)/25

# Defining how to simulate the Training sample and the root of the calibration and test sample

def RVS_TrainSample(n,scale=5):
    return stat.uniform.rvs(loc=0,scale=scale,size=n)

In [ ]:
# 

def ComparisonTransductiveMethods(Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation=True,
                                   RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,scale=5,sigma=0.1,gamma=1,
                                   TrueRegressor=False,fNotNoise=  lambda x : 0,VisualisationSet=False,SaveFig=False):

    fs=16
    ep=3.5
    
    
    
    X_train=RVS_TrainSample(n_train,scale=5)
    y_train=fAndNoise(X_train,sigma)

    
    X_cal1=RVS_TrainSample(n_cal,scale=5)
    X_cal=Psi(X_cal1)
    y_cal=fAndNoise(X_cal1,sigma)

    X_test1=RVS_TrainSample(n_test,scale=5)
    X_test=Psi(X_test1)
    y_test=fAndNoise(X_test1,sigma)

    
    
    if Visualisation:
        


    
        plt.figure(figsize=(10,10))
        plt.rc('text.latex', preamble=r'\usepackage{bm}')
        
        plt.scatter(X_train,y_train,marker='.',color=color_train,label=r"$\mathcal{D}_{{ \mbox{train}}}$ (labeled)",alpha=0.4)
        plt.scatter(X_cal,y_cal,marker='o',color=color_cal,label=r"$\mathcal{D}_{{ \mbox{cal}}}$ (labeled)")
    
        plt.scatter(X_test,y_test,marker='*',color=color_test,zorder=2, s=50,label=r"$\mathcal{D}_{{ \mbox{test}}}$ (Unlabeled)")

        plt.legend(loc=(1.04, 0.25))
        plt.xlabel(r'$X$')
        plt.ylabel(r'$Y$')
        plt.show()

    x_min=min(np.min(X_cal),np.min(X_test))
    x_max=max(np.max(X_cal),np.max(X_test))
    aux=np.linspace(x_min,x_max,10**3)
    auxTriche=aux[np.newaxis].T
    
    
    
    X_trainTriche=X_train[np.newaxis].T
    X_testTriche=X_test[np.newaxis].T
    X_calTriche=X_cal[np.newaxis].T
    X_Label2=np.concatenate((X_cal,X_test))[np.newaxis].T
    y_trainTriche=y_train[np.newaxis].T
    
    
    
    # Transfer learning
    model,loss=jdot_krr(X_trainTriche,y_trainTriche,X_Label2,gamma_g=gamma,numIterBCD = 10,ktype='rbf')
    K_Label2=sklearn.metrics.pairwise.rbf_kernel(X_Label2,X_Label2,gamma=gamma)
    y_pred_Label2=model.predict(K_Label2)
    K_cal=sklearn.metrics.pairwise.rbf_kernel(X_calTriche,X_Label2,gamma=gamma)
    y_pred_cal=model.predict(K_cal).T[0]
    Residual_Distrib=ConformalDistrib(y_pred_cal,y_cal,n_cal,my_score)
    K_test=sklearn.metrics.pairwise.rbf_kernel(X_testTriche,X_Label2,gamma=gamma)
    
    y_pred_test=model.predict(K_test).T[0]    
    CI_OptUp,CI_OptDown,q_classical=ClassicalSCP(y_pred_test,Residual_Distrib,Level)
    
    print("Length of the prediction interval with transfer learning:", 2*q_classical)
    
    ## Without Transfer Learning
    
    
    modelNoTransfer,loss=jdot_krr(X_trainTriche,y_trainTriche,X_trainTriche,gamma_g=gamma,numIterBCD = 10,ktype='rbf')
    K_testNoTransfer=sklearn.metrics.pairwise.rbf_kernel(X_testTriche,X_trainTriche,gamma=gamma)
    y_pred_testNoTransfer=modelNoTransfer.predict(K_testNoTransfer).T[0]

    K_calNoTransfer=sklearn.metrics.pairwise.rbf_kernel(X_calTriche,X_trainTriche,gamma=gamma)
    y_pred_calNoTransfer=modelNoTransfer.predict(K_calNoTransfer).T[0]

    Residual_DistribNoTransfer=ConformalDistrib(y_pred_calNoTransfer,y_cal,n_cal,my_score)
    CI_OptUpNoTransfer,CI_OptDownNoTransfer,q_classicalNoTransfer=ClassicalSCP(y_pred_testNoTransfer,Residual_DistribNoTransfer,Level)
    
    print("Length of the prediction interval without transfer learning:", 2*q_classicalNoTransfer)
    
    
    
    
    ##Without Transfer Learning with D_cal Split
    

    n_cal_train=int(np.ceil(n_cal/2))
    n_cal_cal=n_cal-n_cal_train
    X_cal_train=X_cal[:n_cal_train]
    y_cal_train=y_cal[:n_cal_train]
    X_cal_cal=X_cal[n_cal_train:]
    y_cal_cal=y_cal[n_cal_train:]

    X_cal_trainTriche=X_cal_train[np.newaxis].T
    X_cal_calTriche=X_cal_cal[np.newaxis].T

    y_cal_trainTriche=y_cal_train[np.newaxis].T

    X_Label2Div=np.concatenate((X_cal_cal,X_test))
    X_Label2DivTriche=X_Label2Div[np.newaxis].T
    y_label2Div=np.concatenate((y_cal_cal,y_test))
    y_label2DivTriche=y_label2Div[np.newaxis].T


    modelDiv,loss=jdot_krr(X_cal_trainTriche,y_cal_trainTriche,X_cal_trainTriche,gamma_g=gamma,numIterBCD = 10,ktype='rbf')

    K_cal_cal=sklearn.metrics.pairwise.rbf_kernel(X_cal_calTriche,X_cal_trainTriche,gamma=gamma)
    K_testDiv=sklearn.metrics.pairwise.rbf_kernel(X_testTriche,X_cal_trainTriche,gamma=gamma)

    y_pred_testDiv=modelDiv.predict(K_testDiv).T[0]
    y_pred_cal_cal=modelDiv.predict(K_cal_cal).T[0]



    Residual_DistribDiv=ConformalDistrib(y_pred_cal_cal,y_cal_cal,n_cal_cal,my_score)
    CI_OptUpDiv,CI_OptDownDiv,q_classicalDiv=ClassicalSCP(y_pred_testDiv,Residual_DistribDiv,Level)

    print("Length of the prediction interval without transfer learning and by splitting D_cal:", 2*q_classicalDiv)
    
    
    nTab=np.array([n_cal,n_cal,n_cal_cal])
    mTab=np.array([n_test,n_test,n_test])
    y_predTab=np.array([y_pred_test,y_pred_testNoTransfer,y_pred_testDiv])
    
    ## Graphical representation of prediction intervall
    
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm}')
        
        
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)    
    
    if TrueRegressor:
        auxBis=np.linspace(0,scale,10**4)
        auxBisGraph=Psi(auxBis)
        Regressor=fNotNoise(auxBis)
        plt.plot(auxBisGraph,Regressor,'-',color='k',label=r"True regression function",linewidth=ep)
    
    
    
    if Representation:
        K_aux=sklearn.metrics.pairwise.rbf_kernel(auxTriche,X_Label2,gamma=gamma)
        y_pred_aux=model.predict(K_aux).T[0]
        CI_OptUpaux,CI_OptDownaux,q_classical=ClassicalSCP(y_pred_aux,Residual_Distrib,Level)

        plt.plot(aux,CI_OptUpaux,':',color=colors_blindness[1],label=r'$\bm{\mathcal{I}}^{ \mbox{transfer}}$ (This work)',linewidth=ep)
        plt.plot(aux,CI_OptDownaux,':',color=colors_blindness[1],linewidth=ep)

    
    if RepresentationNoTransfer:
        
        K_auxNoTransfer=sklearn.metrics.pairwise.rbf_kernel(auxTriche,X_trainTriche,gamma=gamma)
        y_pred_auxNoTransfer=modelNoTransfer.predict(K_auxNoTransfer).T[0]

        CI_OptUpauxNoTransfer,CI_OptDownauxNoTransfer, q=ClassicalSCP(y_pred_auxNoTransfer,Residual_DistribNoTransfer,Level)

        plt.plot(aux,CI_OptUpauxNoTransfer,':',color=colors_blindness[2],label=r'$\bm{\mathcal{I}}^{\mbox{naive}}$',linewidth=ep)
        plt.plot(aux,CI_OptDownauxNoTransfer,':',color=colors_blindness[2],linewidth=ep)
    
    if RepresentationDiv:
        
        K_auxDiv=sklearn.metrics.pairwise.rbf_kernel(auxTriche,X_cal_trainTriche,gamma=gamma)
        y_pred_auxDiv=modelDiv.predict(K_auxDiv).T[0]
        CI_OptUpauxDiv,CI_OptDownauxDiv,q_classicalDiv=ClassicalSCP(y_pred_auxDiv,Residual_DistribDiv,Level)
        
        plt.plot(aux,CI_OptUpauxDiv,':',color=colors_blindness[0],label=r'$\bm{\mathcal{I}}^{ \mbox{split}}$',linewidth=ep)
        plt.plot(aux,CI_OptDownauxDiv,':',color=colors_blindness[0],linewidth=ep)

        
    if VisualisationSet:
        plt.scatter(X_train,y_train,marker='.',color=color_train,label=r"$\mathcal{D}_{{ \mbox{train}}}$ (Labeled)",alpha=0.4)
        plt.scatter(X_cal,y_cal,marker='o',color=color_cal,label=r"$\mathcal{D}_{{ \mbox{cal}}}$ (Labeled)")
    
    plt.scatter(X_test,y_test,marker='*',color=color_test,zorder=2, s=50,label=r"$\mathcal{D}_{{ \mbox{test}}}$ (Unlabeled)")
    plt.legend(loc='upper right', fontsize=fs)
    plt.xlabel('x',fontsize=fs)
    plt.ylabel('y',fontsize=fs)
    if SaveFig:
        plt.savefig('TransferCP_'+str(n_train)+'_'+str(n_cal)+'_'+str(n_test)+'.pdf',format='pdf')
    
    plt.show()
    return(np.array([Residual_Distrib,Residual_DistribNoTransfer,Residual_DistribDiv]),nTab,mTab,y_test,y_predTab)

In [ ]:
Level=0.1
n_train=1000
n_cal=30
n_test=20

In [ ]:
ComparisonTransductiveMethods(Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation=True,
                                   RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,scale=5,sigma=0.1,gamma=1,
                                   TrueRegressor=True,fNotNoise=fNotNoise,VisualisationSet=True)

    

In [ ]:
def ComparisonTransductiveMethodsBlackandWhite(Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation=True,
                                   RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,scale=5,sigma=0.1,gamma=1,
                                   TrueRegressor=False,fNotNoise=  lambda x : 0,VisualisationSet=False,SaveFig=False):

    fs=16
    ep=3.5
    
    
    
    X_train=RVS_TrainSample(n_train,scale=5)
    y_train=fAndNoise(X_train,sigma)

    
    X_cal1=RVS_TrainSample(n_cal,scale=5)
    X_cal=Psi(X_cal1)
    y_cal=fAndNoise(X_cal1,sigma)

    X_test1=RVS_TrainSample(n_test,scale=5)
    X_test=Psi(X_test1)
    y_test=fAndNoise(X_test1,sigma)

    
    
    if Visualisation:
        


    
        plt.figure(figsize=(10,10))
        plt.rc('text.latex', preamble=r'\usepackage{bm}')
        
        plt.scatter(X_train,y_train,marker='.',color=color_train,label=r"$\mathcal{D}_{{ \mbox{train}}}$ (labeled)",alpha=0.4)
        plt.scatter(X_cal,y_cal,marker='o',color=color_cal,label=r"$\mathcal{D}_{{ \mbox{cal}}}$ (labeled)")
    
        plt.scatter(X_test,y_test,marker='*',color=color_test,zorder=2, s=50,label=r"$\mathcal{D}_{{ \mbox{test}}}$ (Unlabeled)")

        plt.legend(loc=(1.04, 0.25))
        plt.xlabel(r'$X$')
        plt.ylabel(r'$Y$')
        plt.show()

    x_min=min(np.min(X_cal),np.min(X_test))
    x_max=max(np.max(X_cal),np.max(X_test))
    aux=np.linspace(x_min,x_max,10**3)
    auxTriche=aux[np.newaxis].T
    
    
    
    X_trainTriche=X_train[np.newaxis].T
    X_testTriche=X_test[np.newaxis].T
    X_calTriche=X_cal[np.newaxis].T
    X_Label2=np.concatenate((X_cal,X_test))[np.newaxis].T
    y_trainTriche=y_train[np.newaxis].T
    
    
    
    # Transfer learning
    model,loss=jdot_krr(X_trainTriche,y_trainTriche,X_Label2,gamma_g=gamma,numIterBCD = 10,ktype='rbf')
    K_Label2=sklearn.metrics.pairwise.rbf_kernel(X_Label2,X_Label2,gamma=gamma)
    y_pred_Label2=model.predict(K_Label2)
    K_cal=sklearn.metrics.pairwise.rbf_kernel(X_calTriche,X_Label2,gamma=gamma)
    y_pred_cal=model.predict(K_cal).T[0]
    Residual_Distrib=ConformalDistrib(y_pred_cal,y_cal,n_cal,my_score)
    K_test=sklearn.metrics.pairwise.rbf_kernel(X_testTriche,X_Label2,gamma=gamma)
    
    y_pred_test=model.predict(K_test).T[0]    
    CI_OptUp,CI_OptDown,q_classical=ClassicalSCP(y_pred_test,Residual_Distrib,Level)
    
    print("Length of the prediction interval with Transfer learning:", 2*q_classical)
    
    ## Without Transfer Learning
    
    
    modelNoTransfer,loss=jdot_krr(X_trainTriche,y_trainTriche,X_trainTriche,gamma_g=gamma,numIterBCD = 10,ktype='rbf')
    K_testNoTransfer=sklearn.metrics.pairwise.rbf_kernel(X_testTriche,X_trainTriche,gamma=gamma)
    y_pred_testNoTransfer=modelNoTransfer.predict(K_testNoTransfer).T[0]

    K_calNoTransfer=sklearn.metrics.pairwise.rbf_kernel(X_calTriche,X_trainTriche,gamma=gamma)
    y_pred_calNoTransfer=modelNoTransfer.predict(K_calNoTransfer).T[0]

    Residual_DistribNoTransfer=ConformalDistrib(y_pred_calNoTransfer,y_cal,n_cal,my_score)
    CI_OptUpNoTransfer,CI_OptDownNoTransfer,q_classicalNoTransfer=ClassicalSCP(y_pred_testNoTransfer,Residual_DistribNoTransfer,Level)
    
    print("Length of the prediction interval without Transfer learning:", 2*q_classicalNoTransfer)
    
    
    
    
    ##Without Transfer Learning with D_cal Split
    

    n_cal_train=int(np.ceil(n_cal/2))
    n_cal_cal=n_cal-n_cal_train
    X_cal_train=X_cal[:n_cal_train]
    y_cal_train=y_cal[:n_cal_train]
    X_cal_cal=X_cal[n_cal_train:]
    y_cal_cal=y_cal[n_cal_train:]

    X_cal_trainTriche=X_cal_train[np.newaxis].T
    X_cal_calTriche=X_cal_cal[np.newaxis].T

    y_cal_trainTriche=y_cal_train[np.newaxis].T

    X_Label2Div=np.concatenate((X_cal_cal,X_test))
    X_Label2DivTriche=X_Label2Div[np.newaxis].T
    y_label2Div=np.concatenate((y_cal_cal,y_test))
    y_label2DivTriche=y_label2Div[np.newaxis].T


    modelDiv,loss=jdot_krr(X_cal_trainTriche,y_cal_trainTriche,X_cal_trainTriche,gamma_g=gamma,numIterBCD = 10,ktype='rbf')

    K_cal_cal=sklearn.metrics.pairwise.rbf_kernel(X_cal_calTriche,X_cal_trainTriche,gamma=gamma)
    K_testDiv=sklearn.metrics.pairwise.rbf_kernel(X_testTriche,X_cal_trainTriche,gamma=gamma)

    y_pred_testDiv=modelDiv.predict(K_testDiv).T[0]
    y_pred_cal_cal=modelDiv.predict(K_cal_cal).T[0]



    Residual_DistribDiv=ConformalDistrib(y_pred_cal_cal,y_cal_cal,n_cal_cal,my_score)
    CI_OptUpDiv,CI_OptDownDiv,q_classicalDiv=ClassicalSCP(y_pred_testDiv,Residual_DistribDiv,Level)

    print("Length of the prediction interval without Transfer learning and by splitting D_cal:", 2*q_classicalDiv)
    
    
    nTab=np.array([n_cal,n_cal,n_cal_cal])
    mTab=np.array([n_test,n_test,n_test])
    y_predTab=np.array([y_pred_test,y_pred_testNoTransfer,y_pred_testDiv])
    
    ## Graphical representation of prediction intervall
    
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm}')
        
        
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)    
    
    if TrueRegressor:
        auxBis=np.linspace(0,scale,10**4)
        auxBisGraph=Psi(auxBis)
        Regressor=fNotNoise(auxBis)
        plt.plot(auxBisGraph,Regressor,'-',color='k',label=r"True regression function",linewidth=ep)
    
    
    
    if Representation:
        K_aux=sklearn.metrics.pairwise.rbf_kernel(auxTriche,X_Label2,gamma=gamma)
        y_pred_aux=model.predict(K_aux).T[0]
        CI_OptUpaux,CI_OptDownaux,q_classical=ClassicalSCP(y_pred_aux,Residual_Distrib,Level)

        plt.plot(aux,CI_OptUpaux,'.-',color=colors_blindness[1],label=r'$\bm{\mathcal{I}}^{ \mbox{transfer}}$ (This work)',linewidth=ep)
        plt.plot(aux,CI_OptDownaux,'.-',color=colors_blindness[1],linewidth=ep)

    
    if RepresentationNoTransfer:
        
        K_auxNoTransfer=sklearn.metrics.pairwise.rbf_kernel(auxTriche,X_trainTriche,gamma=gamma)
        y_pred_auxNoTransfer=modelNoTransfer.predict(K_auxNoTransfer).T[0]

        CI_OptUpauxNoTransfer,CI_OptDownauxNoTransfer, q=ClassicalSCP(y_pred_auxNoTransfer,Residual_DistribNoTransfer,Level)

        plt.plot(aux,CI_OptUpauxNoTransfer,':',color=colors_blindness[2],label=r'$\bm{\mathcal{I}}^{\mbox{naive}}$',linewidth=ep)
        plt.plot(aux,CI_OptDownauxNoTransfer,':',color=colors_blindness[2],linewidth=ep)
    
    if RepresentationDiv:
        
        K_auxDiv=sklearn.metrics.pairwise.rbf_kernel(auxTriche,X_cal_trainTriche,gamma=gamma)
        y_pred_auxDiv=modelDiv.predict(K_auxDiv).T[0]
        CI_OptUpauxDiv,CI_OptDownauxDiv,q_classicalDiv=ClassicalSCP(y_pred_auxDiv,Residual_DistribDiv,Level)
        
        plt.plot(aux,CI_OptUpauxDiv,'--',color=colors_blindness[0],label=r'$\bm{\mathcal{I}}^{ \mbox{split}}$',linewidth=ep)
        plt.plot(aux,CI_OptDownauxDiv,'--',color=colors_blindness[0],linewidth=ep)

        
    if VisualisationSet:
        plt.scatter(X_train,y_train,marker='.',color=color_train,label=r"$\mathcal{D}_{{ \mbox{train}}}$ (Labeled)",alpha=0.4)
        plt.scatter(X_cal,y_cal,marker='o',color=color_cal,label=r"$\mathcal{D}_{{ \mbox{cal}}}$ (Labeled)")
    
    plt.scatter(X_test,y_test,marker='*',color=color_test,zorder=2, s=50,label=r"$\mathcal{D}_{{ \mbox{test}}}$ (Unlabeled)")
    plt.legend(loc='upper right', fontsize=fs)
    plt.xlabel('x',fontsize=fs)
    plt.ylabel('y',fontsize=fs)
    if SaveFig:
        plt.savefig('TransferCP_'+str(n_train)+'_'+str(n_cal)+'_'+str(n_test)+'.pdf',format='pdf')
    
    plt.show()
    return(np.array([Residual_Distrib,Residual_DistribNoTransfer,Residual_DistribDiv]),nTab,mTab,y_test,y_predTab)

In [ ]:
ComparisonTransductiveMethodsBlackandWhite(Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation=True,
                                   RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,scale=5,sigma=0.1,gamma=1,
                                   TrueRegressor=True,fNotNoise=fNotNoise,VisualisationSet=True)

    

# Post Hoc bounds for Prediction Interval Task

In [ ]:
def MajSimes(n,m,alpha,delta):
    return np.minimum(np.ceil(m*delta/alpha)-1,m)

In [ ]:
def MajDKWConv(n,m,delta,alpha):
    lamb=IterLambda_Tau(n,m,nIter)(delta)
    I_n_alpha=np.floor((n+1)*alpha)/(n+1)

    return np.minimum(np.ceil(m*(I_n_alpha+lamb))-1,m)

In [ ]:
def AlphaChap(L,n,m,Distrib_Residual):
    T=1-Distrib_Residual.cdf(L/2)
    return T

In [ ]:
def MajPropErrorBoundsAlphaChapComparison(n,m,L,delta,y_pred_test=np.ones(shape=(1,1)),y_test=np.ones(1),Distrib_Residual=np.array([stat.uniform])):
    
    L=np.sort(L)
    l=len(L)
    r=len(Distrib_Residual)
    Alpha_L=np.zeros(shape=(r,l))
    PropMajDKWConv=np.zeros(shape=(r,l))
    print(np.shape(PropMajDKWConv))
    PropMajSimes=np.zeros(shape=(r,l))
    for i in range (r):
        Alpha_L[i]=AlphaChap(L,n[i],m[i],Distrib_Residual[i])
    #print(Alpha_L)
        PropMajSimes[i]=MajSimes(n[i],m[i],delta,Alpha_L[i])/m[i]
    #print(PropMajSimes)
        PropMajDKWConv[i]=MajDKWConv(n[i],m[i],delta,Alpha_L[i])/m[i]
    #print(PropMajDKWConv)
    

    PropErrror=np.ones(shape=(r,l))
    for j in range (r) :
        for i in range (l):
            Level=Alpha_L[j,i]
            (CIClassical_Up,CIClassical_Down,q_classical)=ClassicalSCP(y_pred_test[j],Distrib_Residual[j],Level)
            PropErrror[j,i]=1-NbCoverage(y_test,CIClassical_Up,CIClassical_Down)/m[j]

    print(np.shape(PropErrror))
    print(np.shape(PropMajDKWConv))
    return(PropMajDKWConv,PropMajSimes,PropErrror,Alpha_L)

    


In [ ]:
def GraphTransferPostHoc(L,PropMajDKWConv,PropMajSimes,PropError,Alpha_L,delta,n_train=0,n_cal=0,n_test=0,SaveFig=False):
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')
    ep=2.5
    fs=16
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)
    
    
    plt.plot(L,PropError[0],':',color=colors_blindness[1],label=r"$\mathrm{FCP}\Big(\bm{\mathcal{I}}^{ \mbox{transfer}}\Big)$",linewidth=ep)
    plt.plot(L,PropMajDKWConv[0], color=colors_blindness[1],label=r"${\overline{\mathrm{FCP}}}_\delta^{\small\mbox{ DKW}}\Big(\bm{\mathcal{I}}^{ \mbox{transfer}}\Big)$",linewidth=ep)
    
    plt.plot(L,PropError[1],':',color=colors_blindness[2],label=r"$\mathrm{FCP}\Big(\bm{\mathcal{I}}^{ \mbox{naive}}\Big)$",linewidth=ep)
    plt.plot(L,PropMajDKWConv[1], color=colors_blindness[2],label=r"${\overline{\mathrm{FCP}}}_\delta^{\small\mbox{ DKW}}\Big(\bm{\mathcal{I}}^{ \mbox{naive}}\Big)$",linewidth=ep)
    
    plt.plot(L,PropError[2],':',color=colors_blindness[0],label=r"$\mathrm{FCP}\Big(\bm{\mathcal{I}}^{ \mbox{split}}\Big)$",linewidth=ep)
    plt.plot(L,PropMajDKWConv[2],color=colors_blindness[0],label=r"${\overline{\mathrm{FCP}}}_\delta^{\small\mbox{ DKW}}\Big(\bm{\mathcal{I}}^{ \mbox{split}}\Big)$",linewidth=ep)
    
    deltaString=str(delta)
    plt.xlabel(r"Prediction Interval length $2L$", fontsize=fs)
    plt.ylabel("Error proportion", fontsize=fs)
    plt.legend(loc='upper right', fontsize=fs)
    if SaveFig:
        plt.savefig('Alpha_L_Comparison_'+str(n_train)+'_'+str(n_cal)+'_'+str(n_test)+'.pdf',format='pdf')
    
    plt.show()

In [ ]:
def TransferPostHoc(L,Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation=False,RepresentationNoTransfer=False,RepresentationDiv=False,Visualisation=False,scale=5,sigma=0.1,gamma=1,TrueRegressor=False,fNotNoise=  lambda x : 0,VisualisationSet=True,BlackandWhite=False,SaveFig=False):



    if BlackandWhite:
        Distrib,nTab,mTab,y_test,y_predTab=ComparisonTransductiveMethodsBlackandWhite(Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation,RepresentationNoTransfer,RepresentationDiv,Visualisation,scale,sigma,gamma,TrueRegressor,fNotNoise,VisualisationSet,SaveFig)
    else:
        Distrib,nTab,mTab,y_test,y_predTab=ComparisonTransductiveMethods(Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation,RepresentationNoTransfer,RepresentationDiv,Visualisation,scale,sigma,gamma,TrueRegressor,fNotNoise,VisualisationSet,SaveFig)
    PropMajDKWConv,PropMajSimes,PropError,Alpha_L=MajPropErrorBoundsAlphaChapComparison(nTab,mTab,L,delta,y_predTab,y_test,Distrib)
    
    GraphTransferPostHoc(L,PropMajDKWConv,PropMajSimes,PropError,Alpha_L,delta,n_train,n_cal,n_test,SaveFig)

In [ ]:
L=np.linspace(0.001,2,10**3)
delta=0.2
n_train=1000
n_cal=30
n_test=20

In [ ]:
TransferPostHoc(L,Level,n_train,n_cal,n_test,fAndNoise,Psi,Representation=True,RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,TrueRegressor=True,fNotNoise= fNotNoise,VisualisationSet=True,BlackandWhite=True)

## With Other Function

In [ ]:
# Defining Regression function

#Heteroscedasticity



def fAndNoiseHetero(x,sigma=0.1):
    n=len(x)
    return fNotNoise(x)+sigma*(np.cos(x))*stat.norm.rvs(size=n)

# Definig the transfer function for calibration and test samples

def Psi(x):
    return 0.6*x+x**(2)/25

# Defining how to simulate the Training sample and the root of the calibration and test sample

def RVS_TrainSample(n,scale=5):
    return stat.uniform.rvs(loc=0,scale=scale,size=n)

In [ ]:
n_train=1000
n_cal=300
n_test=200

In [ ]:
TransferPostHoc(L,Level,n_train,n_cal,n_test,fAndNoiseHetero,Psi,Representation=True,RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,TrueRegressor=True,fNotNoise= fNotNoise,VisualisationSet=True)

In [ ]:
# Defining Regression function



def fNotNoise3(x):
    return np.cos(x)-x**2+x**3

def fAndNoise3(x,sigma=0.1):
    n=len(x)
    return fNotNoise3(x)+sigma*stat.norm.rvs(size=n)

# Defining the transfer function for calibration and test samples

def Psi3(x):
    return 0.8*np.log(x)+2*x

# Defining how to simulate the Training sample and the root of the calibration and test sample

def RVS_TrainSample(n,scale=5):
    return stat.uniform.rvs(loc=0,scale=scale,size=n)

In [ ]:
n_train=1000
n_cal=300
n_test=200

L=np.linspace(0.01,20,10**4)

In [ ]:
TransferPostHoc(L,Level,n_train,n_cal,n_test,fAndNoise3,Psi3,Representation=True,RepresentationNoTransfer=True,RepresentationDiv=True,Visualisation=True,TrueRegressor=True,fNotNoise= fNotNoise3,VisualisationSet=True)

# Post Hoc bounds for Novelty Detection Task

## Loading Shuttle Data

In [ ]:
datasetShuttle = fetch_openml(name='shuttle',  as_frame=False)
XShuttle = datasetShuttle.data
yShuttle = datasetShuttle.target.astype(float)

In [ ]:
#test sample 
outlrShuttle, inlrShuttle = XShuttle[np.minimum(yShuttle!=1,1)], XShuttle[yShuttle==1]
m1Shuttle = 300
m0Shuttle = 1500

np.random.shuffle(inlrShuttle)
np.random.shuffle(outlrShuttle)

test1Shuttle, test0Shuttle = outlrShuttle[:m1Shuttle], inlrShuttle[:m0Shuttle]
xShuttle = np.concatenate([test0Shuttle, test1Shuttle]) 
n_testShuttle=m1Shuttle+m0Shuttle
print(m0Shuttle/(m1Shuttle+m0Shuttle))

#NTS
nShuttle=5000
SplitSizeShuttle=3000/5000
n_trainShuttle=int(SplitSizeShuttle*nShuttle)
n_calShuttle=nShuttle-n_trainShuttle


xnullShuttle = inlrShuttle[m0Shuttle:m0Shuttle+nShuttle]



procShuttle = AdaDetectERM(scoring_fn = RandomForestClassifier(max_depth=10),
                                 split_size=SplitSizeShuttle) 
procBatesShuttle=AdaDetectDE(scoring_fn = OCC(scoring_fn = IsolationForest(contamination=0.1)), f0_known = False,
                            split_size=SplitSizeShuttle)


## Compute $\hat{m}_0$

In [ ]:
def m0Adaptatif(delta,pValeurs,n_cal,n_test,Show=True):
    
    
    mm=np.arange(n_test)+1
    
    I_n=np.arange(n_cal+2)/(n_cal+1)
    #Calcul des V_t=Sum(1{p_i>t})
    
    
    V=np.zeros(n_cal+2)
    
    V[0]=n_test
    V[n_cal+1]=0
    
    Lamb=IterLambda_Tau(n_cal,mm,nIter)(delta)
    
    for i in range(1,n_cal+1):
        V[i]=np.sum(pValeurs>I_n[i])
        
        
        
    #m0 for DKW:
    InfEst=np.zeros(n_test)
    Max_rLambda=1*Lamb[0]*np.ones(n_test)
    InfEst[0]=np.min((V[1:n_cal+1]+Max_rLambda[0])/(1-I_n[1:n_cal+1]))
    
    for r in range(2,n_test-1):
        
        Current_rLambda=r*Lamb[r-1]
        if Max_rLambda[r-2]<Current_rLambda:
            Max_rLambda[r-1]=Current_rLambda
        InfEst[r-1]=np.min((V[1:n_cal+1]+Max_rLambda[r-1])/(1-I_n[1:n_cal+1]))
    
    PlusGrand=mm*(InfEst>=mm)
    m0_Hat_DKW=np.max(PlusGrand)
    if  m0_Hat_DKW==0:
         m0_Hat_DKW=n_test
    
    DevDKWBound=Max_rLambda[m0_Hat_DKW-1]
    
    
    #m0 for Simes:
    if delta*(n_cal+1)==np.floor(delta*(n_cal+1)):
        t_max=int(np.floor(delta*(n_cal+1)))
    else:
        t_max=int(np.floor(delta*(n_cal+1)))+1
    
    m0_Hat_Simes=min(n_test,int(np.ceil(np.min(V[1:t_max]/(1-I_n[1:t_max]/delta)))))
    
    if Show:
        print('m0_Hat_DKW:' +str(m0_Hat_DKW))
        print('m0_Hat_Simes:' +str(m0_Hat_Simes))
    
    return (m0_Hat_DKW,m0_Hat_Simes,V,DevDKWBound)

## Post Hoc Procedure

### $p$-value Threshold: t

In [ ]:
def PostHoc_Uniforme_t(delta,x,xnull,procedure,n_cal,n_test,m0,Oracle=False):
    
    
    procedure.fit(x,0.5,xnull)
    
    n=n_cal
    m=n_test
    print('m1:'+str(n_test-m0))
    print('m0:' +str(m0))
    

    FDP=np.zeros(n+2)
    Rejet=np.zeros(n+2)
    
    FDP[n+1]=m0/m
    Rejet[n+1]=m
    
    I_n=np.arange(n+2)/(n+1)
    
    
    pValeurs=np.array([compute_pvalue(x, procedure.null_statistics) for x in procedure.test_statistics])
    
    
    (m0_Hat_DKW,m0_Hat_Simes,V,DevDKWBound)=m0Adaptatif(delta,pValeurs,n_cal,n_test)
    
    
    if Oracle:
        m0_Hat_DKW,m0_Hat_Simes=m0,m0
        DevDKWBound=IterLambda_Tau(n_cal,m0_Hat_DKW,nIter)(delta)
    
    #FDP Computation
    
    for i in range (1,n+2):
        level=i/(n+1)
        Rejet[i]=np.sum(pValeurs<=level)
        FDP[i]=np.sum((pValeurs[:m0]<=level))/max(Rejet[i],1)
    
    
    
    #Post Hoc Bounds
    

    
    Maj_DKW_PostHoc=np.minimum(((m0_Hat_DKW*I_n+DevDKWBound)*(Rejet>0))/(np.maximum(Rejet,1)),1)
     
  
    
    
    Maj_Simes_PostHoc=np.minimum(m0_Hat_Simes*I_n*(Rejet>0)/(np.maximum(Rejet,1)*delta),1)
    
    
    return(FDP,Rejet,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat_DKW,m0_Hat_Simes,pValeurs)

In [ ]:
def ComparisonUniformePostHoc(delta,x,xnull,procedure,procedureBates,n_cal,n_test,m0,m1,t_min=1,Prop_t_max=0.2, VisualisationDouble=True,Oracle=False,SaveFig=False):
    
    FDP,Rejet,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat_DKW,m0_Hat_Simes,pValeurs=PostHoc_Uniforme_t(delta,x,xnull,procedure,n_cal,n_test,m0,Oracle=Oracle)
    FDPBates,RejetBates,Maj_DKW_PostHocBates,Maj_Simes_PostHocBates,m0_Hat_DKWBates,m0_Hat_SimesBates,pValeursBates=PostHoc_Uniforme_t(delta,x,xnull,procedureBates,n_cal,n_test,m0,Oracle=Oracle)
    
    I_n=np.arange(n_cal+2)/(n_cal+1)
    
    t_max=int((n_cal+1)*Prop_t_max)
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')
    ep=2.5
    fs=16
    
    
    plt.plot(I_n[t_min:t_max],Maj_DKW_PostHoc[t_min:t_max],color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{\small \mbox{DKW}}_{t,\delta}:{ \mbox{TwoClass}}$ (This work)",linewidth=ep)
    plt.plot(I_n[t_min:t_max],FDP[t_min:t_max],':',color=colors_blindness[1],label=r"${\mathrm{FDP}}(\mathcal{R}(t)): \mbox{TwoClass}$ (Adaptive score)",linewidth=ep)
    plt.plot(I_n[t_min:t_max],Maj_DKW_PostHocBates[t_min:t_max], color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{\small \mbox{DKW}}_{t,\delta}:{ \mbox{OneClass}}$",linewidth=ep)
    plt.plot(I_n[t_min:t_max],FDPBates[t_min:t_max],':',color=colors_blindness[0],label=r"${\mathrm{FDP}}(\mathcal{R}(t)): \mbox{OneClass}$ (No-Adaptive score)",linewidth=ep)


    
    
    
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)
    
    deltaString=str(delta)
    plt.xlabel(r"Level $t$ of the threshold",fontsize=fs)
    plt.ylabel("Error Proportion",fontsize=fs)
    #plt.title(r"Comparison of $\mathrm{FDP}$ and $\overline{\mathrm{FDP}}_{\delta}$ for a Novelty Detection task with $\delta= $"+deltaString, fontsize=fs)
    plt.legend(fontsize=fs)
    if SaveFig:
        plt.savefig('PostHoc_D_'+str(m1)+'_'+str(m0)+'_'+str(n_cal)+'.pdf',format='pdf')
    plt.show()
    
    if VisualisationDouble:
        plt.figure(figsize=(10,10))
        plt.rc('text.latex', preamble=r'\usepackage{bm} ')
        ep=2.5
        fs=16


        plt.plot(I_n[t_min:t_max],Maj_DKW_PostHoc[t_min:t_max],color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{\small \mbox{DKW}}_{t,\delta}:{ \mbox{TwoClass}}$ (This work)",linewidth=ep)
        plt.plot(I_n[t_min:t_max],FDP[t_min:t_max],':',color=colors_blindness[1],label=r"${\mathrm{FDP}}(\mathcal{R}(t)): \mbox{TwoClass}$ (Adaptive score)",linewidth=ep)
        plt.plot(I_n[t_min:t_max],Maj_DKW_PostHocBates[t_min:t_max], color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{\small \mbox{DKW}}_{t,\delta}:{ \mbox{OneClass}}$",linewidth=ep)
        plt.plot(I_n[t_min:t_max],FDPBates[t_min:t_max],':',color=colors_blindness[0],label=r"${\mathrm{FDP}}(\mathcal{R}(t)): \mbox{OneClass}$ (No-Adaptive score)",linewidth=ep)


        plt.plot(I_n[t_min:t_max],Maj_Simes_PostHoc[t_min:t_max],'--',color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{\small \mbox{Simes}}_{t,\delta}:{ \mbox{TwoClass}}$",alpha=0.3,linewidth=ep)
        plt.plot(I_n[t_min:t_max],Maj_Simes_PostHocBates[t_min:t_max],'--', color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{\small \mbox{Simes}}_{t,\delta}:{ \mbox{OneClass}}$",alpha=0.3,linewidth=ep)

    
    
        for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
            tickLabel.set_fontsize(fs)

        deltaString=str(delta)
        plt.legend(fontsize=fs)
        plt.xlabel(r"Level $t$ of the threshold",fontsize=fs)
        plt.ylabel("Error Proportion",fontsize=fs)
        #plt.title(r"Comparison of $\mathrm{FDP}$ and $\overline{\mathrm{FDP}}_{\delta}$ for a Novelty Detection task with $\delta= $"+deltaString, fontsize=fs)
       
        if SaveFig:
            plt.savefig('PostHoc_D_'+str(m1)+'_'+str(m0)+'_'+str(n_cal)+'_Simes.pdf',format='pdf')
            
        plt.show()

    
    
    
    
    
   
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')
    
    plt.plot(I_n[t_min:t_max],Rejet[t_min:t_max],color=colors_blindness[1],label=r"$\mathcal{R}(\mbox(AD)_\alpha)$",linewidth=ep)
    plt.plot(I_n[t_min:t_max],RejetBates[t_min:t_max],color=colors_blindness[0],label=r"$\mathcal{R}(\mbox(Bates)_\alpha)$",linewidth=ep)
    plt.xlabel(r"Level $\alpha$ of test $\mathrm{FDR}=\alpha$",fontsize=fs)
    plt.ylabel("Number of rejection",fontsize=fs)
    plt.legend(fontsize=fs)
    plt.show()
    
    
    return (FDP,Rejet,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat_DKW,m0_Hat_Simes,FDPBates,RejetBates,Maj_DKW_PostHocBates,Maj_Simes_PostHocBates,m0_Hat_DKWBates,m0_Hat_SimesBates)

In [ ]:
delta=0.2

In [ ]:
ComparisonUniformePostHoc(delta,xShuttle,xnullShuttle,procShuttle,procBatesShuttle,n_calShuttle,n_testShuttle,m0Shuttle,m1Shuttle,t_min=1,Prop_t_max=0.2, VisualisationDouble=True,Oracle=False)

## Ada Detect Procedure and Post Hoc: $\mathrm{FDR}\leq \alpha$

In [ ]:
# Return the FDP, the TDP, and all the PostHoc bounds at level delta for a procedure of novelty detection wich control the FDR at level alpha. 

def Maj_PostHocAdaDetect(delta,x,xnull,procedure,n_cal,n_test,m0,m1,NotAdaptive_m0=False):
    
    procedure.fit(x,0.5,xnull)
    
    n=n_cal
    m=n_test
    print('m0:' +str(m0))
    

    FDP=np.zeros(n+2)
    kBH=np.zeros(n+2)
    TDP=np.zeros(n+2)
    I_n=np.arange(n+2)/(n+1)
    
    
    pValeurs=np.array([compute_pvalue(x, procedure.null_statistics) for x in procedure.test_statistics])
    
    (m0_Hat_DKW,m0_Hat_Simes,V,DevDKWBound)=m0Adaptatif(delta,pValeurs,n_cal,n_test)
    
    #FDP Computation
    
    for i in range (n+2):
        level=i/(n+1)
        RejectionSet=EmpBH(procedure.null_statistics, procedure.test_statistics, level = i/(n+1))
        kBH[i]=len(RejectionSet)
        
        FD=np.sum((RejectionSet<m0))
        FDP[i]=FD/max(kBH[i],1)
        TDP[i]=(kBH[i]-FD)/m1
    
    
    #Post Hoc Bounds
    

    lamb=IterLambda_Tau(n_cal,m0_Hat_DKW,nIter)(delta)
    Maj_DKW_PostHoc=np.minimum((m0_Hat_DKW*I_n*kBH/m+DevDKWBound)*(kBH>0)/(np.maximum(kBH,1)),1)
     
    Maj_Simes_PostHoc=np.minimum(m0_Hat_Simes*I_n*(kBH>0)/(m*delta),1)
    
    if NotAdaptive_m0:
        lamb=IterLambda_Tau(n_cal,m,nIter)(delta)
        Maj_DKW_PostHocNoAdapt=np.minimum((I_n*kBH+m*lamb)*(kBH>0)/(np.maximum(kBH,1)),1)
 
        Maj_Simes_PostHocNoAdapt=np.minimum(I_n*(kBH>0)/(delta),1)
    
        return(FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_DKW_PostHocNoAdapt,Maj_Simes_PostHoc,Maj_Simes_PostHocNoAdapt,m0_Hat_DKW,m0_Hat_Simes)
    
    
    return(FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat_DKW,m0_Hat_Simes)

### Comparison of the performance for One Class Clasification V Two Class Classification

In [ ]:
#Display the graph to compare the effiency of using adpative score or not.

def Graph_PostHocComparison_AdaptiveScores(n_cal,m0,m1,FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat,m0_Hat_Simes,FDPBates,TDPBates,kBHBates,Maj_DKW_PostHocBates,Maj_Simes_PostHocBates,m0_HatBates,m0_Hat_SimesBates,Amin=1,Prop_Amax=0.2,VisualisationDouble=False,SaveFig=False,Subplot_TDP=False):
    
    ep=2.5
    fs=16  
    
    if Subplot_TDP:
        ep=4
        fs=24

    Amax=int((n_cal+1)*Prop_Amax)
    
    if Subplot_TDP:
        plt.figure(1,figsize=(30,15))
    if not Subplot_TDP:
        plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')



    n=n_cal
    
    I_n=np.arange(n+2)/(n+1)
    

    
    if Subplot_TDP:
        plt.subplot(1,2,1)
    
    plt.plot(I_n[Amin:Amax],Maj_DKW_PostHoc[Amin:Amax],color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}:\mbox{TwoClass}$ (This work)",linewidth=ep)
    plt.plot(I_n[Amin:Amax],FDP[Amin:Amax],':',color=colors_blindness[1],label=r"$\mathrm{FDP}(\mbox{AD}_\alpha):\mbox{TwoClass}$",linewidth=ep)
    plt.plot(I_n[Amin:Amax],Maj_DKW_PostHocBates[Amin:Amax], color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}:\mbox{OneClass}$ ",linewidth=ep)
    plt.plot(I_n[Amin:Amax],FDPBates[Amin:Amax],':',color=colors_blindness[0],label=r"$\mathrm{FDP}(\mbox{OneClass}_\alpha):\mbox{OneClass}$",linewidth=ep)
    plt.legend(loc='upper right',fontsize=fs)
    plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
    plt.ylabel("False Discovery Proportion",fontsize=fs)
    
    
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)
    
    if Subplot_TDP:
        plt.subplot(1,2,2)
        
        plt.plot(I_n[Amin:Amax],TDP[Amin:Amax],':',color=colors_blindness[1],label=r"$\mathrm{TDP}(\mbox{AD}_\alpha):\mbox{TwoClass}$",linewidth=ep)
        plt.plot(I_n[Amin:Amax],TDPBates[Amin:Amax],':',color=colors_blindness[0],label=r"$\mathrm{TDP}(\mbox{OneClass}_\alpha):\mbox{OneClass}$",linewidth=ep) 
        
        plt.legend(loc='upper right',fontsize=fs)   
        plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
        plt.ylabel("True Discovery Proportion",fontsize=fs)
    
    
        for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
            tickLabel.set_fontsize(fs)
    
    if SaveFig:
        plt.savefig('PostHoc_FDR_D_'+str(m1)+'_'+str(m0)+'_'+str(n_cal)+'.pdf',format='pdf')
    
    plt.show()
 
    
    

    if VisualisationDouble:
        
        
        
        if Subplot_TDP:
            plt.figure(2,figsize=(30,15))
        if not Subplot_TDP:
            plt.figure(figsize=(10,10))
            
        plt.rc('text.latex', preamble=r'\usepackage{bm} ')

        if Subplot_TDP:
            plt.subplot(1,2,1)
        
        plt.plot(I_n[Amin:Amax],Maj_DKW_PostHoc[Amin:Amax],color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}:\mbox{TwoClass}$ (This work)",linewidth=ep)
        plt.plot(I_n[Amin:Amax],FDP[Amin:Amax],':',color=colors_blindness[1],label=r"$\mathrm{FDP}(\mbox{AD}_\alpha):\mbox{TwoClass}$",linewidth=ep)
        plt.plot(I_n[Amin:Amax],Maj_DKW_PostHocBates[Amin:Amax], color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}:\mbox{OneClass}$ ",linewidth=ep)
        plt.plot(I_n[Amin:Amax],FDPBates[Amin:Amax],':',color=colors_blindness[0],label=r"$\mathrm{FDP}(\mbox{OneClass}_\alpha):\mbox{OneClass}$",linewidth=ep)

        
        plt.plot(I_n[Amin:Amax],Maj_Simes_PostHoc[Amin:Amax],'--',color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{Simes}_{\delta}(\mbox{AD}_\alpha):\mbox{TwoClass}$",alpha=0.3,linewidth=ep)
        plt.plot(I_n[Amin:Amax],Maj_Simes_PostHocBates[Amin:Amax],'--', color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{Simes}_{\delta}(\mbox{OneClass}_\alpha):\mbox{OneClass}$",alpha=0.3,linewidth=ep)
    

        plt.legend(loc='upper right',fontsize=fs)
        plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
        plt.ylabel("False Discovery Proportion",fontsize=fs)
       
        for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
            tickLabel.set_fontsize(fs)
    
    
        if Subplot_TDP:
            plt.subplot(1,2,2)

            plt.plot(I_n[Amin:Amax],TDP[Amin:Amax],':',color=colors_blindness[1],label=r"$\mathrm{TDP}(\mbox{AD}_\alpha):\mbox{TwoClass}$",linewidth=ep)
            plt.plot(I_n[Amin:Amax],TDPBates[Amin:Amax],':',color=colors_blindness[0],label=r"$\mathrm{TDP}(\mbox{OneClass}_\alpha):\mbox{OneClass}$",linewidth=ep) 

            plt.legend(loc='upper right',fontsize=fs)   
            plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
            plt.ylabel("True Discovery Proportion",fontsize=fs)

            for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
                tickLabel.set_fontsize(fs)

            
            
        if SaveFig:
            plt.savefig('PostHoc_FDR_D_'+str(m1)+'_'+str(m0)+'_'+str(n_cal)+'_Simes.pdf',format='pdf')

        plt.show()
    
    
    
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')
    
    plt.plot(I_n[Amin:Amax],kBH[Amin:Amax],color=colors_blindness[1],label=r"$\mathcal{R}(\mbox(AD)_\alpha):\mbox{TwoClass}$",linewidth=ep)
    plt.plot(I_n[Amin:Amax],kBHBates[Amin:Amax],color=colors_blindness[0],label=r"$\mathcal{R}(\mbox{OneClass}_\alpha):\mbox{OneClass}$",linewidth=ep)
    plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
    plt.ylabel("Number of rejection",fontsize=fs)
    plt.legend(fontsize=fs)
    plt.show()

In [ ]:
def Comparison_AdaptiveScores(delta,x,xnull,procedure,procedureBates,n_cal,m1,m0,
                              Amin=1,Prop_Amax=0.2,
                              VisualisationDouble=False,SaveFig=False,Subplot_TDP=False):
    
    
    n_test=m1+m0
    FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat_DKW,m0_Hat_Simes=Maj_PostHocAdaDetect(delta,x,xnull,procedure,n_cal,n_test,m0,m1)
    FDPBates,TDPBates,kBHBates,Maj_DKW_PostHocBates,Maj_Simes_PostHocBates,m0_Hat_DKWBates,m0_Hat_SimesBates=Maj_PostHocAdaDetect(delta,x,xnull,procedureBates,n_cal,n_test,m0,m1)
    
    Graph_PostHocComparison_AdaptiveScores(n_cal,m0,m1,FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_Simes_PostHoc,m0_Hat_DKW,m0_Hat_Simes,FDPBates,TDPBates,kBHBates,Maj_DKW_PostHocBates,Maj_Simes_PostHocBates,m0_Hat_DKWBates,m0_Hat_SimesBates,Amin,Prop_Amax,VisualisationDouble,SaveFig,Subplot_TDP)

In [ ]:
Comparison_AdaptiveScores(delta,xShuttle,xnullShuttle,procShuttle,procBatesShuttle,n_calShuttle,m1Shuttle,m0Shuttle,
                              Amin=1,Prop_Amax=0.2,
                              VisualisationDouble=True,Subplot_TDP=False)

In [ ]:
Comparison_AdaptiveScores(delta,xShuttle,xnullShuttle,procShuttle,procBatesShuttle,n_calShuttle,m1Shuttle,m0Shuttle,
                              Amin=1,Prop_Amax=0.2,
                              VisualisationDouble=True,Subplot_TDP=True)

### Effect of using $\hat{m}_0$ instead of $m$ in the PostHoc bounds

In [ ]:
def Graph_PostHoc_Comparison_m0Hat(n_cal,m0,m1,delta,FDP,Maj_DKW_PostHoc,Maj_DKW_PostHocNoAdapt,Maj_Simes_PostHoc,Maj_Simes_PostHocNoAdapt,Amin=1,Prop_Amax=0.2,SaveFig=False):
    ep=2.5
    fs=16  
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')
    
    
    Amax=int((n_cal+1)*Prop_Amax)
    n=n_cal
    I_n=np.arange(n+2)/(n+1)
    
    plt.plot(I_n[Amin:Amax],FDP[Amin:Amax],':',color=colors_blindness[1],label=r"$\mathrm{FDP}(\mbox{AD}_\alpha)$",linewidth=ep)
    

    plt.plot(I_n[Amin:Amax],Maj_DKW_PostHoc[Amin:Amax],color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}$ with $m_0$ estimation",linewidth=ep)
    plt.plot(I_n[Amin:Amax],Maj_Simes_PostHoc[Amin:Amax],'--',color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{Simes}_{\delta}(\mbox{AD}_\alpha)$ with $m_0$ estimation",linewidth=ep)
        
        
    plt.plot(I_n[Amin:Amax],Maj_DKW_PostHocNoAdapt[Amin:Amax],color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}$ without $m_0$ estimation",linewidth=ep)
    plt.plot(I_n[Amin:Amax],Maj_Simes_PostHocNoAdapt[Amin:Amax],'--',color=colors_blindness[0],label=r"$\overline{\mathrm{FDP}}^{Simes}_{\delta}(\mbox{AD}_\alpha)$ without $m_0$ estimation",linewidth=ep)
    
    plt.legend(loc='upper right',fontsize=fs)
    plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
    plt.ylabel("False Discovery Proportion",fontsize=fs)
       
        
        
        
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)

    if SaveFig:
        plt.savefig('PostHoc_m0Adapt_D_'+str(m1)+'_'+str(m0)+'_'+str(n_cal)+'_Simes.pdf',format='pdf')

    plt.show()

In [ ]:
def Comparison_m0Hat(delta,x,xnull,procedure,n_cal,m1,m0,
                              Amin=1,Prop_Amax=0.2,SaveFig=False):
    
    n_test=m0+m1
    
    (FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_DKW_PostHocNoAdapt,Maj_Simes_PostHoc,Maj_Simes_PostHocNoAdapt,m0_Hat_DKW,m0_Hat_Simes)=Maj_PostHocAdaDetect(delta,x,xnull,procedure,n_cal,n_test,m0,m1,NotAdaptive_m0=True)
    
    Graph_PostHoc_Comparison_m0Hat(n_cal,m0,m1,delta,FDP,Maj_DKW_PostHoc,Maj_DKW_PostHocNoAdapt,Maj_Simes_PostHoc,Maj_Simes_PostHocNoAdapt,Amin,Prop_Amax,SaveFig)

In [ ]:
Comparison_m0Hat(delta,xShuttle,xnullShuttle,procShuttle,n_calShuttle,m1Shuttle,m0Shuttle,
                              Amin=1,Prop_Amax=0.2)

### Effect of $\delta$ in the PostHoc bounds

In [ ]:
def Maj_PostHocAdaDetect_Diff_Delta(Delta,x,xnull,procedure,n_cal,n_test,m0,m1):
    
    procedure.fit(x,0.5,xnull)
    
    N=len(Delta)
    n=n_cal
    m=n_test
    print('m0:' +str(m0))
    

    FDP=np.zeros(n+2)
    kBH=np.zeros(n+2)
    TDP=np.zeros(n+2)
    I_n=np.arange(n+2)/(n+1)
    
    Maj_DKW_PostHoc=np.zeros((N,n+2))
    Maj_Simes_PostHoc=np.zeros((N,n+2))
    
    
    pValeurs=np.array([compute_pvalue(x, procedure.null_statistics) for x in procedure.test_statistics])
    
    
    
    
    for i in range (n+2):
        level=i/(n+1)
        RejectionSet=EmpBH(procedure.null_statistics, procedure.test_statistics, level = i/(n+1))
        kBH[i]=len(RejectionSet)
        
        FD=np.sum((RejectionSet<m0))
        FDP[i]=FD/max(kBH[i],1)
        TDP[i]=(kBH[i]-FD)/m1
    
    for j in range (N):
        delta=Delta[j]
    
        (m0_Hat_DKW,m0_Hat_Simes,V,DevDKWBound)=m0Adaptatif(delta,pValeurs,n_cal,n_test,False)
        #Post Hoc Bounds
        
        Maj_DKW_PostHoc[j]=np.minimum((m0_Hat_DKW*I_n*kBH/m+DevDKWBound)*(kBH>0)/(np.maximum(kBH,1)),1)
        Maj_Simes_PostHoc[j]=np.minimum(m0_Hat_Simes*I_n*(kBH>0)/(m*delta),1)

       


    return(FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_Simes_PostHoc)

In [ ]:
def Graph_PostHoc_Comparison_Delta(n_cal,m0,m1,Delta,FDP,Maj_DKW_PostHoc,Maj_Simes_PostHoc,Amin=1,Prop_Amax=0.2,Comparison_02=False,SaveFig=False):
    ep=2.5
    fs=16  
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} ')
    
    
    Amax=int((n_cal+1)*Prop_Amax)
    n=n_cal
    N=len(Delta)
    I_n=np.arange(n+2)/(n+1)
    
    if Comparison_02:
        N=N-1
    
    plt.plot(I_n[Amin:Amax],FDP[Amin:Amax],':',color=colors_blindness[1],label=r"$\mathrm{FDP}(\mbox{AD}_\alpha)$",linewidth=ep)
    
    for j in range (N):
        deltaString=str(Delta[j])
        plt.plot(I_n[Amin:Amax],Maj_DKW_PostHoc[j,Amin:Amax],color=colors_blindness[j],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}$. $\delta=$"+deltaString,linewidth=ep)
        plt.plot(I_n[Amin:Amax],Maj_Simes_PostHoc[j,Amin:Amax],'--',color=colors_blindness[j],label=r"$\overline{\mathrm{FDP}}^{Simes}_{\delta}(\mbox{AD}_\alpha)$. $\delta=$"+deltaString,linewidth=ep)
        
    if Comparison_02:
        deltaString=str(Delta[N])
        plt.plot(I_n[Amin:Amax],Maj_DKW_PostHoc[N,Amin:Amax],color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{DKW}_{\alpha,\delta}$. $\delta=$"+deltaString,linewidth=ep)
        plt.plot(I_n[Amin:Amax],Maj_Simes_PostHoc[N,Amin:Amax],'--',color=colors_blindness[1],label=r"$\overline{\mathrm{FDP}}^{Simes}_{\delta}(\mbox{AD}_\alpha)$. $\delta=$"+deltaString,linewidth=ep)
        
        
    plt.legend(loc='upper right',fontsize=fs)
    plt.xlabel(r"Target $\mathrm{FDR}$ level $\alpha$",fontsize=fs)
    plt.ylabel("False Discovery Proportion",fontsize=fs)
       
        
        
        
    for tickLabel in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
        tickLabel.set_fontsize(fs)

    if SaveFig:
        plt.savefig('PostHoc_Delta_D_'+str(m1)+'_'+str(m0)+'_'+str(n_cal)+'_Simes.pdf',format='pdf')

    plt.show()
   

In [ ]:
def Comparison_Delta(Delta,x,xnull,procedure,n_cal,m1,m0,Comparison_02=False,
                              Amin=1,Prop_Amax=0.2,SaveFig=False):
    n_test=m1+m0
    
    if Comparison_02:
        Delta=np.append(Delta,0.2)
    
    FDP,TDP,kBH,Maj_DKW_PostHoc,Maj_Simes_PostHoc=Maj_PostHocAdaDetect_Diff_Delta(Delta,x,xnull,procedure,n_cal,n_test,m0,m1)
    
    Graph_PostHoc_Comparison_Delta(n_cal,m0,m1,Delta,FDP,Maj_DKW_PostHoc,Maj_Simes_PostHoc,Amin,Prop_Amax,Comparison_02,SaveFig)

In [ ]:
Delta=np.array([0.05])

In [ ]:
Comparison_Delta(Delta,xShuttle,xnullShuttle,procShuttle,n_calShuttle,m1Shuttle,m0Shuttle,
                              Amin=1,Prop_Amax=0.2,Comparison_02=True)

In [ ]:
DeltaBis=np.array([0.05,0.1,0.9])

In [ ]:
Comparison_Delta(DeltaBis,xShuttle,xnullShuttle,procShuttle,n_calShuttle,m1Shuttle,m0Shuttle,
                              Amin=1,Prop_Amax=0.2)

## Experiment with different data set

### Credit Card Data Set

In [ ]:
datasetCreditCard = fetch_openml(name='creditcard', version=1, as_frame=False)
XCreditCard = datasetCreditCard.data
yCreditCard = datasetCreditCard.target.astype(float)

In [ ]:
#test sample 
outlrCreditCard, inlrCreditCard = XCreditCard[yCreditCard==1], XCreditCard[yCreditCard==0]

np.random.shuffle(outlrCreditCard)
np.random.shuffle(inlrCreditCard)

m1CreditCard = 260
m0CreditCard = 500
test1CreditCard, test0CreditCard = outlrCreditCard[:m1CreditCard], inlrCreditCard[:m0CreditCard]
xCreditCard = np.concatenate([test0CreditCard, test1CreditCard]) 
n_testCreditCard=m1CreditCard+m0CreditCard
print(m0CreditCard/(m1CreditCard+m0CreditCard))
#NTS
nCreditCard=3000
SplitSizeCreditCard=2000/3000
n_trainCreditCard=int(SplitSizeCreditCard*nCreditCard)
n_calCreditCard=nCreditCard-n_trainCreditCard


xnullCreditCard = inlrCreditCard[m0CreditCard:m0CreditCard+nCreditCard]


procCreditCard = AdaDetectERM(scoring_fn = RandomForestClassifier(max_depth=10),
                                 split_size=SplitSizeCreditCard) 
procBatesCreditCard=AdaDetectDE(scoring_fn = OCC(scoring_fn = IsolationForest(contamination=0.1)), f0_known = False,
                            split_size=SplitSizeCreditCard)


In [ ]:
Comparison_AdaptiveScores(delta,xCreditCard,xnullCreditCard,procCreditCard,procBatesCreditCard,n_calCreditCard,m1CreditCard,m0CreditCard,
                              Amin=1,Prop_Amax=0.2,
                              VisualisationDouble=True,Subplot_TDP=True)

In [ ]:
Comparison_m0Hat(delta,xCreditCard,xnullCreditCard,procCreditCard,n_calCreditCard,m1CreditCard,m0CreditCard,
                              Amin=1,Prop_Amax=0.2)

In [ ]:
Comparison_Delta(Delta,xCreditCard,xnullCreditCard,procCreditCard,n_calCreditCard,m1CreditCard,m0CreditCard,
                              Amin=1,Prop_Amax=0.2,Comparison_02=True)

### Mammography Data

In [ ]:
datasetMam = fetch_openml(name='mammography', version=1, as_frame=False)
XMam = datasetMam.data
yMam = datasetMam.target.astype(float)

In [ ]:
#test sample 
outlrMam, inlrMam = XMam[yMam==1], XMam[yMam==-1]

np.random.shuffle(outlrMam)
np.random.shuffle(inlrMam)



m1Mam = 260
m0Mam = 500



test1Mam, test0Mam = outlrMam[:m1Mam], inlrMam[:m0Mam]
xMam = np.concatenate([test0Mam, test1Mam]) 
n_testMam=m1Mam+m0Mam
print(m0Mam/(m1Mam+m0Mam))
#NTS
nMam=3000
SplitSizeMam=2000/3000
n_trainMam=int(SplitSizeMam*nMam)
n_calMam=nMam-n_trainMam


xnullMam = inlrMam[m0Mam:m0Mam+nMam]



procMam = AdaDetectERM(scoring_fn = RandomForestClassifier(max_depth=10),
                                 split_size=SplitSizeMam) 
procBatesMam=AdaDetectDE(scoring_fn = OCC(scoring_fn = IsolationForest(contamination=0.1)), f0_known = False,
                            split_size=SplitSizeMam)

In [ ]:
Comparison_AdaptiveScores(delta,xMam,xnullMam,procMam,procBatesMam,n_calMam,m1Mam,m0Mam,
                              Amin=1,Prop_Amax=0.2,
                              VisualisationDouble=True,Subplot_TDP=True)

In [ ]:
Comparison_m0Hat(delta,xMam,xnullMam,procMam,n_calMam,m1Mam,m0Mam,
                              Amin=1,Prop_Amax=0.2)

In [ ]:
Comparison_Delta(Delta,xMam,xnullMam,procMam,n_calMam,m1Mam,m0Mam,
                              Amin=1,Prop_Amax=0.2,Comparison_02=True)

# Tightness of the DKW-type Bound

## Comparison between $\mathbb{P}\left(\sup_{t\in[0,1]}\left(\hat{F}_m(t)-I_n(t)\right) >\lambda\right)$ and $B^{{{ DKW}}}(\lambda,n,m)$.

In [ ]:
## Compute the Bound

def DKW_Full(n,m):
    def Bound_DKW(lamb):
        t_nm=(n*m)/(n+m)
        Num=2*np.sqrt(2*np.pi)*lamb*t_nm
        Den=np.sqrt(n+m)
        return np.minimum((1+Num/Den)*np.exp(-2*t_nm*lamb**2),1)
    return Bound_DKW

In [ ]:
n=500
m=100
N=10**4
l=np.linspace(0.01,0.25,10**3)

In [ ]:
def Show_Tightness_DKW_Full(n,m,l,N,SaveFig=False):
    
    
    ep=2.5
    fs=16  
    
    
    plt.figure(figsize=(10,10))
    plt.rc('text.latex', preamble=r'\usepackage{bm} \usepackage{amsfonts} ')
    
    B_DKW=DKW_Full(n,m)(l)
    pValues=Simu_EmpiricalPvalues(n,m,N)
    Prop=np.ones(m)/m
    nn=(np.arange(n+1)+1)/(n+1)
    TabMaxCDF=np.zeros(N)
    for i in range (N):
        F_m=stat.rv_discrete(values=np.array([pValues[i],Prop])).cdf(nn)
        TabMaxCDF[i]=np.max((F_m-nn))
        #print(TabMaxCDF)
    r=len(l)
    CourbesPropRespectDKW=np.zeros(r)
    for i in range (r):
        CourbesPropRespectDKW[i]=np.sum(TabMaxCDF>l[i])/N
    plt.plot(l,CourbesPropRespectDKW,color=colors_blindness[0],label=r'Estimation of $\mathbb{P}\left(\sup_{t\in[0,1]}\left(\hat{F}_m(t)-I_n(t)\right) >\lambda\right)$',linewidth=ep)
    plt.plot(l,B_DKW,color=colors_blindness[1],label=r"$B^{{{ DKW}}}(\lambda,n,m)$",linewidth=ep)
    plt.xlabel(r"$\lambda$",fontsize=fs)
    plt.title(r" n= "+str(n)+ " and m= "+str(m),fontsize=18)
    plt.legend(fontsize=fs)
    
    
    if SaveFig:
        plt.savefig('Tightness_DKW_Full'+'_'+str(n)+'_'+str(m)+'.pdf',format='pdf')
    
    plt.show()

In [ ]:
Show_Tightness_DKW_Full(n,m,l,N)

In [ ]:
Show_Tightness_DKW_Full(n,500,l,N)

In [ ]:
Show_Tightness_DKW_Full(1000,500,l,N)

In [ ]:
Show_Tightness_DKW_Full(100,100,l,N)